In [11]:
import umap
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering, DBSCAN
from sklearn.metrics import normalized_mutual_info_score
from sklearn.preprocessing import MinMaxScaler, normalize, StandardScaler 

### Import packages

In [12]:
#@title Scaling and Normalization Funcs
#@markdown - max_min_scale(df)
#@markdown - z_score_scale(df)
#@markdown - robust_scale(df)
#@markdown - zero_replace(df)
#@markdown - norm_replace(df)
#@markdown - log_trans(df)
def max_min_scale(df):
    df_mm = df.copy()
    max_v = df_mm.describe().iloc[-1]
    min_v = df_mm.describe().iloc[3]
    return (df_mm - min_v) / (max_v-min_v)


def z_score_scale(df):
    df_zs = df.copy()
    mean_v = df_zs.describe().iloc[1]
    std_v = df_zs.describe().iloc[2]
    return (df_zs - mean_v) / std_v


def robust_scale(df):
    df_rb = df.copy()
    Q1 = df_rb.describe().iloc[-4]
    Q2 = df_rb.describe().iloc[-3]
    Q3 = df_rb.describe().iloc[-2]
    return (df_rb - Q2) / (Q3-Q1)


def zero_replace(df):
    df_rp = df.copy()
    for c in tqdm(df_rp.columns):
        f = df_rp[c]
        des = f.describe()
        mean = des['mean']
        for i in range(len(df)):
            if df_rp[c].iloc[i] == 0:
                df_rp[c].iloc[i] = mean
    return df_rp


def norm_replace(df,mean=True):
    df_rp = df.copy()
    for c in tqdm(df_rp.columns):
        f = df_rp[c]
        des = f.describe()
        mean = des['mean']
        std = des['std']
        upper = stats.norm.ppf(0.9999,loc=mean,scale=std)
        lower = stats.norm.ppf(0.0001,loc=mean,scale=std)
        # cot = 0
        for i in range(len(df)):
            if df_rp[c].iloc[i] > upper:
                # cot+= 1
                if mean:
                    df_rp[c].iloc[i] = mean
                else:
                    df_rp[c].iloc[i] = upper
            if df_rp[c].iloc[i] < lower:
                # cot += 1
                if mean:
                    df_rp[c].iloc[i] = mean
                else:
                    df_rp[c].iloc[i] = lower
        # print("For column %s, replace %d vars"%(c,cot))
    return df_rp


def log_trans(df):
    df_log = df.copy()
    return np.log2(df_log)




In [13]:
#@title Tools Funcs
#@markdown - evaluation(y_true,y_pred)
#@markdown - feature_visualization(df,save_path)
def evaluation(y_true,y_pred):
    return normalized_mutual_info_score(y_true, y_pred, average_method='geometric')

def feature_visualization(df,save_path):
    fea_num = len(df.columns)
    for i in range(fea_num):
        plt.subplot()
        fig, ax = plt.subplots()
        sns.distplot(df['f%d'%(i+1)],bins=50, kde=False, rug=False,ax=ax)
        fig.savefig('%s/%d.png' % (save_path,i+1))

#### Gene Data
1.   Read csv

In [14]:
df = pd.read_csv('genedata.csv').set_index('id')
label = df['class']
df.drop(columns=['class'],inplace=True)

2. Replace outliers



In [15]:
df_rp = norm_replace(df, False)

100%|██████████| 7000/7000 [02:04<00:00, 56.22it/s]


3. Scaling and normalization

In [16]:
df_norm_mm = max_min_scale(df_rp)
df_norm_rb = robust_scale(df_rp)
df_norm_zs = z_score_scale(df_rp)
df_norm_zr = zero_replace(df_rp)
df_norm_log = log_trans(df_norm_zr)

clustering = KMeans(n_clusters=5).fit(df_norm_mm).labels_
print("max_min_scale",evaluation(label,clustering))
clustering = KMeans(n_clusters=5).fit(df_norm_rb).labels_
print("robust_scale",evaluation(label,clustering))
clustering = KMeans(n_clusters=5).fit(df_norm_zs).labels_
print("z-score norm", evaluation(label,clustering))
clustering = KMeans(n_clusters=5).fit(df_norm_log).labels_
print("log trans", evaluation(label,clustering))
clustering = KMeans(n_clusters=5).fit(df_rp).labels_
print("original", evaluation(label,clustering))

100%|██████████| 7000/7000 [01:08<00:00, 101.61it/s]
max_min_scale 0.818735466632994
robust_scale 0.7811054029445896
z-score norm 0.7733640130591249
log trans 0.8793419121329562
original 0.8608676761524444


4. Feature Selection and Clustering

In [ ]:
def test_various_clustering(feature):
  clrs=[AgglomerativeClustering(5),KMeans(5)]
  clrs_n=['Agg','KM']
  res=[]
  for i,clr in enumerate(clrs):
    y_pred = clr.fit(feature).labels_
    res.append((clrs_n[i],evaluation(label,y_pred)))
  return res

def test_DBSCAN(feature):
  best_m = 0
  best_a = (0,0)
  for e in range(1,100,5):
    for ms in range(10,300,10):
      y_pred = DBSCAN(eps=e,min_samples=ms).fit(feature).labels_
      m = evaluation(label,y_pred)
      if m > best_m and m <= 1:
        best_m = m
        best_a = (e,ms)
  return (best_m,best_a)

res_table = []
ds_name = ['ori','mm','rb','zs','log']
for i,ds in enumerate([df_rp,df_norm_mm,df_norm_rb,df_norm_zs,df_norm_log]):
  for fs in ['PCA','UMAP','UMAP-corr','UMAP-bray']:
    best_sum = 0
    for d in range(2,100):
      if fs == 'PCA':
        df_red = PCA(n_components=d).fit_transform(ds)
      if fs == 'UMAP':
        df_red = umap.UMAP(n_components=d).fit_transform(ds)
      if fs == 'UMAP-corr':
        df_red = umap.UMAP(n_components=d,metric='correlation').fit_transform(ds)
      if fs == 'UMAP-bray':
        df_red = umap.UMAP(n_components=d,metric='braycurtis').fit_transform(ds)
      m = test_various_clustering(df_red)
      tmp = sum([i[1] for i in m])
      if tmp > best_sum:
        best_sum = tmp
        best_d = d
        best_m = m
    print(ds_name[i],fs,best_d,best_m)
    res_table.append([ds_name[i],fs,best_d,best_m])
    best_db = 0
    for d in range(100,701,100):
      if fs == 'PCA':
        df_red = PCA(n_components=d).fit_transform(ds)
      if fs == 'UMAP':
        df_red = umap.UMAP(n_components=d).fit_transform(ds)
      if fs == 'UMAP-corr':
        df_red = umap.UMAP(n_components=d,metric='correlation').fit_transform(ds)
      if fs == 'UMAP-bray':
        df_red = umap.UMAP(n_components=d,metric='braycurtis').fit_transform(ds)
      db_m = test_DBSCAN(df_red)
      m = ('DBSCAN',db_m[0],db_m[1])
      tmp = m[1]
      if tmp > best_db:
        best_db = tmp
        best_d = d
        best_m = m
    print(ds_name[i],fs,best_d,best_m)
    res_table.append([(ds_name[i],fs,best_d,best_m)])

for res in res_table:
  print(res)

ori PCA 20 [('Agg', 0.9561656224554782), ('KM', 0.8545905317828452)]
ori PCA 100 ('DBSCAN', 0.7225812957771073, (61, 30))
ori UMAP 41 [('Agg', 0.9897857835205175), ('KM', 0.9897857835205175)]
ori UMAP 100 ('DBSCAN', 0.9897857835205175, (1, 10))
ori UMAP-corr 52 [('Agg', 0.9897857835205175), ('KM', 0.9897857835205175)]
ori UMAP-corr 600 ('DBSCAN', 0.9897857835205175, (1, 10))
ori UMAP-bray 5 [('Agg', 0.9845357442209692), ('KM', 0.9897977296624982)]
ori UMAP-bray 200 ('DBSCAN', 0.9845357442209692, (1, 10))
mm PCA 65 [('Agg', 0.9376158014880337), ('KM', 0.8158447913483609)]
mm PCA 600 ('DBSCAN', 0.5607117482804441, (11, 60))
mm UMAP 22 [('Agg', 0.9894927609750906), ('KM', 0.9894927609750906)]
mm UMAP 200 ('DBSCAN', 0.9894927609750905, (1, 10))
mm UMAP-corr 2 [('Agg', 0.9906346094268289), ('KM', 0.9906346094268289)]
mm UMAP-corr 400 ('DBSCAN', 0.9906346094268288, (1, 10))
mm UMAP-bray 86 [('Agg', 0.9745826607222169), ('KM', 0.9745826607222169)]
mm UMAP-bray 300 ('DBSCAN', 0.974144013974997

The best method is

In [ ]:
df_umap = umap.UMAP(n_components=18,metric='correlation',random_state=1).fit_transform(df_norm_log)
clustering =  AgglomerativeClustering(n_clusters=5).fit(df_umap).labels_
print(evaluation(label,clustering))

0.9947412140458939


In [ ]:
np.savetxt('solution_gene.txt',clustering,fmt= "%d")

In [17]:
top_res = []
for i in range(100):
  df_red = umap.UMAP(n_components=18,metric='correlation').fit_transform(df_norm_log)
  y_pred = AgglomerativeClustering(5).fit(df_red).labels_
  top_res.append(evaluation(label,y_pred))
print(sum(top_res)/100)
sec_res = []
for i in range(100):
  df_red = umap.UMAP(n_components=400,metric='correlation').fit_transform(df_norm_log)
  y_pred = DBSCAN(eps=1,min_samples=10).fit(df_red).labels_
  sec_res.append(evaluation(label,y_pred))
print(sum(sec_res)/100)

0.9925647135977889


0.946462503963869


### MS Data

1. Read CSV

In [4]:
df = pd.read_csv('msdata.csv').set_index('id')
label = df['class']
df.drop(columns=['class'],inplace=True)

2. Apply log transformation and max-min scalling

In [5]:
#@title Apply different kinds of preprocessing
# df_norm_mm = max_min_scale(df) #improved
# df_norm_rb = robust_scale(df)
# df_norm_zs = z_score_scale(df)
# df_norm_zr = zero_replace(df)
# df_norm_nm = norm_replace(df)
df_norm_log = np.log(df)
df_log_mm = MinMaxScaler().fit_transform(df_norm_log)

3. Feature Selection and Clustering

In [ ]:
def test_various_clustering(feature):
  clrs=[AgglomerativeClustering(3),KMeans(3)]
  clrs_n=['Agg','KM']
  res=[]
  for i,clr in enumerate(clrs):
    y_pred = clr.fit(feature).labels_
    res.append((clrs_n[i],evaluation(label,y_pred)))
  return res

def test_DBSCAN(feature):
  best_m = 0
  best_a = (0,0)
  for e in range(1,100,5):
    for ms in range(10,300,10):
      y_pred = DBSCAN(eps=e,min_samples=ms).fit(feature).labels_
      m = evaluation(label,y_pred)
      if m > best_m and m <= 1:
        best_m = m
        best_a = (e,ms)
  return (best_m,best_a)

res_table = []
ds_name = ['ori','log_mm']
for i,ds in enumerate([df,df_log_mm]):
  for fs in ['PCA','UMAP','UMAP-bray']:
    best_sum = 0
    for d in range(1,101):
      if fs == 'PCA':
        df_red = PCA(n_components=d).fit_transform(ds)
      if fs == 'UMAP':
        df_red = umap.UMAP(n_components=d,random_state=1).fit_transform(ds)
      if fs == 'UMAP-bray':
        df_red = umap.UMAP(n_components=d,metric='braycurtis',random_state=1).fit_transform(ds)
      m = test_various_clustering(df_red)
      tmp = sum([i[1] for i in m])
      if tmp > best_sum:
        best_sum = tmp
        best_d = d
        best_m = m
    print(ds_name[i],fs,best_d,best_m)
    res_table.append([ds_name[i],fs,best_d,best_m])
    best_db = 0
    for d in range(100,601,100):
      if fs == 'PCA':
        df_red = PCA(n_components=d).fit_transform(ds)
      if fs == 'UMAP':
        df_red = umap.UMAP(n_components=d,random_state=1).fit_transform(ds)
      if fs == 'UMAP-bray':
        df_red = umap.UMAP(n_components=d,metric='braycurtis',random_state=1).fit_transform(ds)
      db_m = test_DBSCAN(df_red)
      m = ('DBSCAN',db_m[0],db_m[1])
      tmp = m[1]
      if tmp > best_db:
        best_db = tmp
        best_d = d
        best_m = m
    print(ds_name[i],fs,best_d,best_m)
    res_table.append([(ds_name[i],fs,best_d,best_m)])

for res in res_table:
  print(res)



ori PCA 3 [('Agg', 0.6680182026290474), ('KM', 0.6709632127122674)]
ori PCA 200 ('DBSCAN', 0.6312943959326814, (91, 30))
ori UMAP 85 [('Agg', 0.0071549603402983955), ('KM', 0.0009572384541853534)]
ori UMAP 100 ('DBSCAN', 0.25, (1, 50))
ori UMAP-bray 88 [('Agg', 0.9732452436433033), ('KM', 0.9566711210343127)]
ori UMAP-bray 500 ('DBSCAN', 0.9683805744400898, (1, 90))
log_mm PCA 97 [('Agg', 0.5735824116472192), ('KM', 0.8570943466135686)]
log_mm PCA 100 ('DBSCAN', 0.6227186648828904, (6, 30))
log_mm UMAP 2 [('Agg', 0.00658892979164847), ('KM', 0.0011034617989326607)]
log_mm UMAP 100 ('DBSCAN', 0.25, (1, 40))
log_mm UMAP-bray 51 [('Agg', 1.0), ('KM', 0.9907595759943276)]
log_mm UMAP-bray 200 ('DBSCAN', 0.9617936982531473, (1, 100))
['ori', 'PCA', 3, [('Agg', 0.6680182026290474), ('KM', 0.6709632127122674)]]
[('ori', 'PCA', 200, ('DBSCAN', 0.6312943959326814, (91, 30)))]
['ori', 'UMAP', 85, [('Agg', 0.0071549603402983955), ('KM', 0.0009572384541853534)]]
[('ori', 'UMAP', 100, ('DBSCAN', 0.

The best result is 

In [ ]:
df_umap = umap.UMAP(n_components=51,metric='braycurtis',random_state=1).fit_transform(df_log_mm)
clustering =  AgglomerativeClustering(n_clusters=3).fit(df_umap).labels_
print(evaluation(label,clustering))

1.0


In [ ]:
np.savetxt('solution_ms.txt',clustering,fmt= "%d")

Due to the randomness of the UMAP, we tried to find the best result with different random_state,Also to prove that our result will not be affected so much by the randomness, we calculated the average NMI values over 100 experiements. 

In [8]:
top_res = []
top_res_km = []
for i in range(1,100):
  df_red = umap.UMAP(n_components=51,metric='braycurtis').fit_transform(df_log_mm)
  y_pred = AgglomerativeClustering(3).fit(df_red).labels_
  y_pred_km = KMeans(3).fit(df_red).labels_
  top_res.append(evaluation(label,y_pred))
  top_res_km.append(evaluation(label,y_pred_km))

In [10]:
print(np.mean(top_res))
print(np.mean(top_res_km))
print(min(top_res))
print(min(top_res_km))

0.9557222137223342
0.9537300573197666
0.9223693549975279
0.9263952978284072
